In [3]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from inspect_ai.log import read_eval_log, EvalLog

# set the log path
log_path = os.path.join(os.getcwd(), '..', 'logs/')


In [12]:
log_file_path = log_path + f"openai/gpt-4.1-mini/2025-06-09T22-02-56-04-00_alignment-eval_DpkheRmoBg5sgXhZyvobP3.eval"
log: EvalLog = read_eval_log(log_file_path)

In [17]:
log.stats

EvalStats(started_at='2025-06-09T22:02:56-04:00', completed_at='2025-06-09T22:06:27-04:00', model_usage={'openai/gpt-4.1-mini': ModelUsage(input_tokens=248292, output_tokens=100265, total_tokens=348557, input_tokens_cache_write=None, input_tokens_cache_read=0, reasoning_tokens=0)})